<a href="https://colab.research.google.com/github/shubhamphal/One-NLP/blob/master/Detecting_Contradiction_and_Entailment_in_Multi_lingual_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow_text

     |████████████████████████████████| 2.6MB 8.5MB/s 


In [6]:
!pip install --upgrade tensorflow

     |████████████████████████████████| 320.4MB 53kB/s 
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [10]:
 !pip install -q kaggle

In [11]:
 !mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [12]:
!cp kaggle.json ~/.kaggle/

In [13]:
!chmod 600 ~/.kaggle/kaggle.json

In [14]:
!kaggle competitions download -c contradictory-my-dear-watson

  0% 0.00/1.23M [00:00<?, ?B/s]
100% 1.23M/1.23M [00:00<00:00, 82.7MB/s]
  0% 0.00/66.0k [00:00<?, ?B/s]
100% 66.0k/66.0k [00:00<00:00, 59.9MB/s]
  0% 0.00/536k [00:00<?, ?B/s]
100% 536k/536k [00:00<00:00, 171MB/s]


In [15]:
!pip install transformers

     |████████████████████████████████| 1.4MB 8.4MB/s 
     |████████████████████████████████| 890kB 29.5MB/s 
     |████████████████████████████████| 2.9MB 40.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=9b26c2f85e20c8f3ec8eefd1dd0c42fec5ae855ad785ba040729b464ceb60d2b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [23]:
!pip install tokenizers

In [24]:
!pip3 install sentencepiece

     |████████████████████████████████| 1.1MB 9.6MB/s 


In [16]:

import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import tensorflow_text
import os
import sys
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, GlobalMaxPooling1D, Bidirectional, LSTM, GRU, GlobalMaxPool1D, Dropout, Lambda, Concatenate
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from sklearn.metrics import roc_auc_score
from nltk.corpus import stopwords
from keras.optimizers import Adam
from nltk.tokenize import word_tokenize

In [17]:
import zipfile
with zipfile.ZipFile('test.csv.zip', 'r') as zip_ref:
    zip_ref.extractall(os.getcwd())
with zipfile.ZipFile('train.csv.zip', 'r') as zip_ref:
    zip_ref.extractall(os.getcwd())

In [26]:
import torch
from transformers import BertTokenizer,BertForSequenceClassification,AdamW,get_linear_schedule_with_warmup
import tensorflow as tf
import os
import pandas as pd
import numpy as np

In [27]:
gpu=tf.test.gpu_device_name()
print(gpu)

/device:GPU:0


In [28]:
if torch.cuda.is_available():    
    device=torch.device("cuda")
    print('There are %d GPU(s) available.' %torch.cuda.device_count())
    print('We will use the GPU:',torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [29]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [30]:
test.id

0       c6d58c3f69
1       cefcc82292
2       e98005252c
3       58518c10ba
4       c32b0d16df
           ...    
5190    5f90dd59b0
5191    f357a04e86
5192    1f0ea92118
5193    0407b48afb
5194    16c2f2ab89
Name: id, Length: 5195, dtype: object

In [31]:
train.groupby('language')['id'].count()

language
Arabic         401
Bulgarian      342
Chinese        411
English       6870
French         390
German         351
Greek          372
Hindi          374
Russian        376
Spanish        366
Swahili        385
Thai           371
Turkish        351
Urdu           381
Vietnamese     379
Name: id, dtype: int64

In [32]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [33]:
seed=2
tokenized_train=tokenizer.batch_encode_plus(train.iloc[:,1:3].to_numpy().tolist(),max_length=128,pad_to_max_length=True,return_tensors='pt')
labels_train=torch.tensor(train.label.values[:])
tokenized_val=tokenizer.batch_encode_plus(train.iloc[-1000:,1:3].to_numpy().tolist(),max_length=128,pad_to_max_length=True,return_tensors='pt')
labels_val=torch.tensor(train.label.values[-1000:])
tokenized_test=tokenizer.batch_encode_plus(test.iloc[:,1:3].to_numpy().tolist(),max_length=128,pad_to_max_length=True,return_tensors='pt')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [34]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

batch_size=32
train_data=TensorDataset(tokenized_train['input_ids'],tokenized_train['attention_mask'],tokenized_train['token_type_ids'],labels_train)
train_sampler=RandomSampler(train_data)
train_dataloader=DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
val_data=TensorDataset(tokenized_val['input_ids'],tokenized_val['attention_mask'],tokenized_val['token_type_ids'],labels_val)
val_sampler=RandomSampler(val_data)
val_dataloader=DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)
test_data=TensorDataset(tokenized_test['input_ids'],tokenized_test['attention_mask'],tokenized_test['token_type_ids'])
test_dataloader=DataLoader(test_data, batch_size=batch_size)

In [35]:
model=BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',num_labels=3,output_attentions=False,output_hidden_states=False)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [36]:
optimizer=AdamW(model.parameters(),lr=2e-5)
epochs=8


total_steps=len(train_dataloader)*epochs

scheduler=get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps = total_steps)

In [37]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [38]:
import random

seed_val = 12

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []

for epoch_i in range(0, epochs):
    model.train()
    total_loss=0
    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_attention_mask=batch[1].to(device)
        b_token_type = batch[2].to(device)
        b_labels = batch[3].to(device)

    
        outputs=model(b_input_ids,token_type_ids=b_token_type,attention_mask=b_attention_mask,labels=b_labels)
        
        loss = outputs[0]
        total_loss += loss.item()

        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))

    model.eval()

   
    val_accuracy=0
    nb_val_steps=0

   
    for batch in val_dataloader:
        
        
        batch = tuple(t.to(device) for t in batch)
        
        b_input_ids,b_attention_mask, b_token_type, b_labels = batch
        
        with torch.no_grad():        
            outputs = model(b_input_ids,token_type_ids=b_token_type,attention_mask=b_attention_mask)
        
        logits = outputs[0]
        
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        tmp_val_accuracy = flat_accuracy(logits, label_ids)
        val_accuracy += tmp_val_accuracy
        nb_val_steps += 1

   
    print("  Accuracy: {0:.2f}".format(val_accuracy/nb_val_steps))


  Average training loss: 0.92
  Accuracy: 0.70

  Average training loss: 0.67
  Accuracy: 0.86

  Average training loss: 0.45
  Accuracy: 0.92

  Average training loss: 0.30
  Accuracy: 0.96

  Average training loss: 0.19
  Accuracy: 0.98

  Average training loss: 0.13
  Accuracy: 0.99

  Average training loss: 0.09
  Accuracy: 1.00

  Average training loss: 0.07
  Accuracy: 1.00


In [39]:
final_output = []
for batch in test_dataloader:
  #Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  b_input_ids,b_attention_mask, b_token_type= batch
  
  
  with torch.no_grad():        
      outputs = model(b_input_ids,token_type_ids=b_token_type,attention_mask=b_attention_mask)
  
  logits = outputs[0]
  final_output.extend(np.argmax(logits.detach().cpu().numpy(), axis=1).flatten())
  

In [40]:
output = pd.DataFrame({'id': test.id,
                       'prediction': final_output})
output.to_csv('submission.csv', index=False)